In [21]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('iteration4_skun276').getOrCreate()

from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [5]:
df = spark.read.load('./telco_data_modified_final.csv', format="csv", header="true")

In [4]:
df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [4]:
df.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [5]:
df.describe().show()

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7042|     

In [6]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [4]:
df.count()

7043

In [7]:
df.na.drop().count()

6955

In [6]:
df.na.drop().show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [23]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType, BooleanType, NumericType, DoubleType)

In [5]:
df.select('Churn','Contract','InternetService','tenure','TotalCharges','PaperlessBilling').show()

+-----+--------------+---------------+------+------------+----------------+
|Churn|      Contract|InternetService|tenure|TotalCharges|PaperlessBilling|
+-----+--------------+---------------+------+------------+----------------+
|   No|Month-to-month|            DSL|     1|       29.85|             Yes|
|   No|      One year|            DSL|    34|      1889.5|              No|
|  Yes|Month-to-month|            DSL|     2|      108.15|             Yes|
|   No|      One year|            DSL|    45|     1840.75|              No|
|  Yes|Month-to-month|    Fiber optic|     2|      151.65|             Yes|
|  Yes|Month-to-month|    Fiber optic|     8|       820.5|             Yes|
|   No|Month-to-month|    Fiber optic|    22|      1949.4|             Yes|
|   No|          null|            DSL|    10|       301.9|              No|
|  Yes|Month-to-month|    Fiber optic|    28|     3046.05|             Yes|
|   No|      One year|            DSL|    62|     3487.95|              No|
|   No|Month

In [6]:
df2 = spark.read.load('./722_it4_dataset.csv', format="csv", header="true")

In [18]:
df2.columns

['Customer satisfaction rate', 'customerID']

In [12]:
extraColumn = df2.select('Customer satisfaction rate')

In [20]:
#df.withColumn(extraColumn).show()

TypeError: withColumn() missing 1 required positional argument: 'col'

In [24]:
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

for i in ["TotalCharges","Churn"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df = pipeline.fit(df).transform(df).withColumn(unlist(i+"_Scaled")).drop(i+"_Vect")

Py4JJavaError: An error occurred while calling o422.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9, localhost, executor driver): java.lang.NumberFormatException: For input string: "No"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:272)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at org.apache.spark.sql.execution.datasources.csv.CSVTypeCast$.castTo(CSVInferSchema.scala:252)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:125)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:94)
	at org.apache.spark.sql.execution.datasources.csv.C

In [11]:
#LINEAR REGRESSION

lr = LinearRegression(featuresCol ='tenure', labelCol ='Churn', predictionCol='prediction')

df = df.withColumn("tenure", df["tenure"].cast(IntegerType()))

In [12]:
data_schema = [StructField('Churn',IntegerType(),True),
               StructField('tenure',IntegerType(),True),
                StructField('customerID',IntegerType(),True),
                StructField('gender',StringType(),True),
                StructField('Partner',IntegerType(),True),
                StructField('Dependents',IntegerType(),True),
                StructField('PhoneService',IntegerType(),True),
                StructField('MultipleLines',IntegerType(),True),
                StructField('DeviceProtection',IntegerType(),True),
                StructField('StreamingTV',IntegerType(),True),
                StructField('StreamingMovies',IntegerType(),True),
                StructField('PaymentMethod',IntegerType(),True),
                StructField('SeniorCitizen',IntegerType(),True),
                StructField('TotalCharges',IntegerType(),True),
                StructField('TechSupport',IntegerType(),True),
               
               StructField('OnlineSecurity',IntegerType(),True),
                StructField('OnlineBackup',IntegerType(),True),
                StructField('PaperlessBilling',IntegerType(),True),
                StructField('Contract',IntegerType(),True),
                StructField('InternetService',IntegerType(),True)
              ]

final_struct = StructType(fields=data_schema)

df = spark.read.load('./telco_data_modified_final.csv', format="csv", header="true", schema=final_struct)

df.printSchema()

root
 |-- Churn: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- customerID: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- Partner: integer (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- PhoneService: integer (nullable = true)
 |-- MultipleLines: integer (nullable = true)
 |-- DeviceProtection: integer (nullable = true)
 |-- StreamingTV: integer (nullable = true)
 |-- StreamingMovies: integer (nullable = true)
 |-- PaymentMethod: integer (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- TotalCharges: integer (nullable = true)
 |-- TechSupport: integer (nullable = true)
 |-- OnlineSecurity: integer (nullable = true)
 |-- OnlineBackup: integer (nullable = true)
 |-- PaperlessBilling: integer (nullable = true)
 |-- Contract: integer (nullable = true)
 |-- InternetService: integer (nullable = true)



In [18]:
df_model = df.select('tenure', 'Churn')

In [19]:
# data_schema = [StructField('Churn',BooleanType(),True),
#                StructField('tenure',IntegerType(),True)]

# final_struct = StructType(fields=data_schema)

# df = df.withColumn("tenure", df["tenure"].cast(NumericType()))



list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_model.select(df_model['Churn'],
    list_to_vector_udf(df_model["tenure"]).alias("tenure")
)


# _rdd = df.rdd.map(lambda row:row[0])
# temp_rdd = df.rdd.map(lambda row:row[1])
# new_df = city_rdd.zip(temp_rdd.map(lambda x:Vectors.dense(x))).toDF(schema=['city','temperatures'])


In [20]:
lrModel = lr.fit(df_with_vectors)

Py4JJavaError: An error occurred while calling o108.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): java.lang.NumberFormatException: For input string: "Female"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:272)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:29)
	at org.apache.spark.sql.execution.datasources.csv.CSVTypeCast$.castTo(CSVInferSchema.scala:252)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:125)
	at org.apache.spark.sql.execution.datasources.csv.CSVRelation$$anonfun$csvParser$3.apply(CSVRelation.scala:94)
	at org.apache.spark.sql.execution.datasources.c

In [9]:
####################################################################################################

#LOGISTIC REGRESSION

logreg = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial", featuresCol ='Partner', labelCol ='Churn')


In [10]:
#logregModel = logreg.fit(df_with_vectors)

logregModel = logreg.fit(df)
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

IllegalArgumentException: 'requirement failed: Column Partner must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually StringType.'